# Imports 

In [101]:
import pandas as pd
import  numpy as np
import os
from random import randint, choice
from sklearn import linear_model
from joblib import dump, load

In [4]:
df = pd.read_csv('data/iris.csv', sep=",", index_col=None, header=None)

# Linear Model Data

In [66]:
def random_char():    
    return randint(97, 122)

model_prefix = [str.upper(chr(s)) for s in range(97,107)]

In [67]:
model_prefix

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

In [73]:
def generate_data(model_prefix='A', number_of_records = 500,number_of_features = 2):
    train_X = np.random.randint(number_of_records, size=(number_of_records, number_of_features))
    train_y = np.random.randint(number_of_records, size=number_of_records)
    #val_X = np.random.randint(number_of_records, size=(number_of_records, number_of_features))
    #val_y = np.random.randint(number_of_records, size=number_of_records)        
    df_train = pd.concat([pd.DataFrame(train_X),pd.DataFrame(train_y)],axis=1)
    #df_train.insert(value=chr(random_char()) + chr(random_char()) + chr(random_char()),loc=0,column='model_type')    
    df_train.insert(value=model_prefix,loc=0,column='model_type')    
    return df_train

In [74]:
dfs=[]
for i in range(10):
    dfs.append(generate_data(model_prefix[i]))

In [75]:
df_final=pd.concat(dfs)

In [76]:
df_final.model_type.value_counts()

A    500
H    500
G    500
C    500
E    500
I    500
D    500
F    500
J    500
B    500
Name: model_type, dtype: int64

In [77]:
df_final.to_csv("data/train_multi_model.csv", sep=",", index=None,header=False)

# Do models

In [99]:
prefix_model_path = 'data/models'
g=df_final.groupby(by=df_final.columns[0])

In [102]:
for model_name, tmp_df in g:    
    train_y = tmp_df.iloc[:, 1]
    train_X = tmp_df.iloc[:, 2:]
    reg = linear_model.BayesianRidge(n_iter=300, normalize=True)
    tmp_model = reg.fit(train_X, train_y)
    model_file_path = os.path.join(prefix_model_path, "regression_{}.joblib".format(model_name))
    dump(reg, model_file_path)

# Simple predict

In [106]:
reg.predict(np.array([[12,13]]))[0]

248.6238256232875

# Predict  with model input prefix - Endpoint simulation

In [107]:
input_row = np.array([['A',12,13]])

In [114]:
model_prefix = input_row[0][0]
print(model_prefix)
tmp_model_path = os.path.join(prefix_model_path, "regression_{}.joblib".format(model_prefix))
print(tmp_model_path)
new_model = load(tmp_model_path)
print(new_model)

A
data/models/regression_A.joblib
BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=True, tol=0.001, verbose=False)


In [129]:
input_row_data = np.array(list(input_row[0,1:]),dtype=np.float64).reshape(1,2)
input_row_data

array([[12., 13.]])

In [130]:
new_model.predict(input_row_data)

array([252.99151015])